<a href="https://colab.research.google.com/github/lawlete/Varios/blob/main/Resume_Art_Grandes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tarea de Generación de Texto:**

Usa Python y un modelo de lenguaje como GPT (puedes usar la API de OpenAI o Hugging Face). Escribe un script que genere un resumen de un artículo largo sobre un tema específico (por ejemplo, “Avances en IA y aprendizaje automático en 2025”). El resumen debe ser claro y contener los puntos clave del artículo original.

**El siguiente script Resume un articulo sobre tecnologias e IA extenso**
**Se usa un LLM y la biblioteca Lanchchain para acceder a las funciones de:
  -Carga del articulo extenso
  -Fragmentacion del articulo
  -Almacenamiento
  -Resumen de los fragmentos
  -Resumen final de todo el contenido**

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install tiktoken


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
import tiktoken

**Articulo sobre: "Avances en IA y aprendizaje automático en 2025"**

**https://baufest.com/el-futuro-de-la-ia-y-la-computacion-en-la-nube-tendencias-para-2025-y-mas-alla/**


In [ ]:
def Carga_Art(url):
    # 1. Cargar el artículo (puede ser un archivo de texto o URL)

    url = url

    # Instancia el loader
    loader_url = WebBaseLoader(url)
    documents_url = loader_url.load()  # Devuelve una lista de documentos

    return documents_url

def divide_en_fragemtos(documents, chunk_size=1000, chunk_overlap=200):
    # Divide el documento en fragmentos manejables
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)

    return docs

def resumen(docs, chain):
    # Ejecutar la cadena para generar el resumen del artículo
    summary = chain.run(docs)

    return summary

def imprime_url_ejemplos():
  urls = [
      ("https://www.bbc.com/mundo/articles/c4gxzx0kpp6o", "Análisis de las expectativas en IA para 2025 según BBC Mundo."),
      ("https://www.computerworld.es/article/3631423/como-sera-el-avance-de-la-inteligencia-artificial-generativa-en-2025.html", "Crecimiento y aplicaciones de la IA generativa en 2025."),
      ("https://codelabsacademy.com/es/blog/artificial-intelligence-trends-in-2025-whats-next-in-ai", "Tendencias emergentes en IA y su impacto en la investigación científica."),
      ("https://baufest.com/el-futuro-de-la-ia-y-la-computacion-en-la-nube-tendencias-para-2025-y-mas-alla/", "Integración de la IA en la computación en la nube."),
      ("https://es.weforum.org/stories/2025/01/ia-tecnologia-y-la-era-inteligente-en-davos-2025-lo-que-hay-que-saber/", "Discusiones sobre IA en el Foro Económico Mundial de Davos 2025."),
      ("https://www.ironhack.com/es/blog/avances-en-inteligencia-artificial-una-mirada-2024", "Avances esperados en IA y su impacto económico."),
      ("https://bimsoluciones.com/5-tendencias-ia-para-2025-que-no-puedes-perderte/", "Cinco tendencias clave en IA para 2025."),
      ("https://www.next-step.es/tendencias-y-retos-de-la-ia-en-2025/", "Desafíos y oportunidades en el desarrollo de la IA en 2025."),
      ("https://sergio.ec/ia-2025-el-futuro-que-ya-esta-aqui/", "Transformaciones de la IA en educación y aprendizaje."),
      ("https://cadenaser.com/comunitat-valenciana/2025/02/05/la-empresa-prompsit-del-pcumh-participa-en-un-proyecto-sobre-modelos-de-lengua-masivos-de-codigo-abierto-para-una-ia-transparente-en-europa-radio-elche/", "Proyecto europeo de modelos de lenguaje abiertos y transparentes.")
  ]

  for url, description in urls:
      print(f"{description}\nURL: {url}\n")





In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
# Verifica y carga el file con las variables de entorno si existe
if load_dotenv(find_dotenv()):
    if "OPENAI_API_KEY" in os.environ: # Verifica que existe la variable OPENAI_API_KEY
        print("OPENAI cargada") # Imprime el mensaje Si existe la variable de OpenAI
    else:
        print("No se cargo la api_key de openai, debera cargarla manualmente")
else:
      print("No se cargo la api_key, debera cargarla manualmente")

OPENAI cargada


In [ ]:
def main():

  # Instanciamos el LLM con un modelo adecuado para resumen
  model = "gpt-4-turbo"  # Tambien podemos usar "gpt-4-turbo" o "gpt-3.5-turbo"
  llm = ChatOpenAI(model=model,
               temperature=0,
                ) # la API KEY la toma de las variables de entorno, del file .env

  # Gener el System Prompt para la fase MAP (resumen de cada fragmento)
  map_prompt_template = """Eres un experto en tecnología y tu tarea es resumir artículos extensos de manera clara, concisa,
  y contener los puntos clave del artículo original.
  Lee el siguiente fragmento de un artículo y proporciona un resumen detallado en español:

  {text}

  Resumen:"""

  map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

  # Crea el System Prompt para la fase REDUCE (unión de los resúmenes parciales)
  reduce_prompt_template = """Eres un especialista en inteligencia artificial y tecnología.
  Tienes los resúmenes de diferentes fragmentos de un artículo y debes combinarlos en un único resumen coherente.
  El resumen final debe ser claro, preciso, completo y contener los puntos clave del artículo original.
  Organizar resumen final en español, colocando los titulos principales en negritas, luego los subtitulos en negritas con letra mas chica,
  y en los puntos principales en cursiva.

  Resúmenes parciales:
  {text}

  Resumen final:"""

  reduce_prompt_1 = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])

# Otra opcion mas completa de system prompts
  # Crea el System Prompt para la fase REDUCE (unión de los resúmenes parciales)
  reduce_prompt_template = """Eres un especialista en inteligencia artificial y tecnología.
  Tienes los resúmenes de diferentes fragmentos de un artículo y debes combinarlos en un único resumen coherente.

  "Por favor, resuma el texto proporcionado en los fragmentos en un formato estructurado con los siguientes elementos:

  - Un párrafo de introducción que contextualice el tema.
  - Títulos de sección para cada categoría de tendencias (por ejemplo, 'Tendencias de IA', 'Tendencias de Computación en la Nube', etc.).
  - Enumeración de las tendencias dentro de cada sección, con un título breve y una descripción concisa.
  - Un párrafo de conclusión que resuma los puntos clave.

  Utiliza asteriscos (*) para destacar los títulos de sección y números para enumerar las tendencias.
  Asegúrate de mantener un tono coherente, claro, sin repeteciones y conciso en todo el texto."

  Resúmenes parciales:
  {text}

  Resumen final:"""

  reduce_prompt_2 = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])



  # Configurar la cadena de resumen con el enfoque MAP-REDUCE
    # El prompt de sistema se incorpora en la cadena de resumen.
    # LangChain ya tiene configurada una cadena de resumen "map_reduce" que realiza:
    #   - Map: Resumir cada fragmento individualmente.
    #   - Reduce: Combinar esos resúmenes en un resumen final.
  chain = load_summarize_chain(
      llm,
      chain_type="map_reduce",
      map_prompt=map_prompt,
      combine_prompt=reduce_prompt_2,
      verbose=True
  )

# Define la URL a cargar

# Ejemplo de url, por defecto se usa la siguiente, sino de ingresa otra
  url = "https://baufest.com/el-futuro-de-la-ia-y-la-computacion-en-la-nube-tendencias-para-2025-y-mas-alla/"


  imprime_url_ejemplos()
  print("Ingrese la url del articulo que deseaa resumir (puede copiar y pegar algunas de las url que se listan arriba): ", "\n")
  print("Tambien puede pulsar <Enter> y se resumira la siguiente url: ", "\n")
  print(url)
  input_url = input()
  if input_url:
    url = input_url


# Carga de la url en un objeto documents de langchian
  documents = Carga_Art(url)

# Dividir el documento en fragmentos
  docs = divide_en_fragemtos(documents, chunk_size=1000, chunk_overlap=200)

  # Resumen el articulo
  summary = resumen(docs, chain)

  # Mostrar el resumen final en pantalla
  print("Resumen del artículo:", "\n")
  print("url: ", url, "\n\n")
  print(summary)


if __name__ == "__main__":
    main()

Análisis de las expectativas en IA para 2025 según BBC Mundo.
URL: https://www.bbc.com/mundo/articles/c4gxzx0kpp6o

Crecimiento y aplicaciones de la IA generativa en 2025.
URL: https://www.computerworld.es/article/3631423/como-sera-el-avance-de-la-inteligencia-artificial-generativa-en-2025.html

Tendencias emergentes en IA y su impacto en la investigación científica.
URL: https://codelabsacademy.com/es/blog/artificial-intelligence-trends-in-2025-whats-next-in-ai

Integración de la IA en la computación en la nube.
URL: https://baufest.com/el-futuro-de-la-ia-y-la-computacion-en-la-nube-tendencias-para-2025-y-mas-alla/

Discusiones sobre IA en el Foro Económico Mundial de Davos 2025.
URL: https://es.weforum.org/stories/2025/01/ia-tecnologia-y-la-era-inteligente-en-davos-2025-lo-que-hay-que-saber/

Avances esperados en IA y su impacto económico.
URL: https://www.ironhack.com/es/blog/avances-en-inteligencia-artificial-una-mirada-2024

Cinco tendencias clave en IA para 2025.
URL: https://bim